# Processing updated migration matrix
This notebook takes the 2017 bilateral migration matrix from World Bank and formats it for the Schon, Johnson `netlm` model.

In [54]:
import pandas as pd
from fuzzywuzzy import process, fuzz

In [307]:
df = pd.read_csv('/Users/brandonrose/repos/WM/columbia/MR-QAP-refugees/Data_5.csv')

ccode_lookup = dict(zip(df.ccode1,df['state.origin.name']))
country_lookup = dict(zip(df['state.origin.name'],df.ccode1))

ccode_lookup_str = {}
for kk, vv in ccode_lookup.items():
    ccode_lookup_str[str(kk)] = vv
    
mig = pd.read_csv('../refugee_data/bilateralmigrationmatrix20172018.csv')[:214]

In [308]:
mig_cols = set(mig.columns) - set(['Other North','Other South','World'])
schon_cols = set(df['state.origin.name'])

In [309]:
mig_schon = {}
for i in mig_cols:
    match = process.extractBests(i, schon_cols)
    if match[0][1]>60:
        mig_schon[i] = match[:2]
    else:
        mig_schon[i]= None

In [310]:
mig_schon['New Caledonia'] = None
mig_schon['American Samoa'] = None
mig_schon['West Bank and Gaza'] = None
mig_schon['St. Martin (French part)'] = None
mig_schon['Kyrgyz Republic'] = [('Kyrgyzstan',100)]
mig_schon['Channel Islands'] = None
mig_schon['Slovak Republic'] = 'Slovakia'
mig_schon['Aruba'] = None
mig_schon['Curacao'] = None
mig_schon['Turks and Caicos Islands'] = None
mig_schon['Hong Kong SAR, China'] = None
mig_schon['Northern Mariana Islands'] = None
mig_schon['Virgin Islands (U.S.)'] = None
mig_schon['Faeroe Islands'] = None
mig_schon['Guam'] = None
mig_schon['Congo, Dem. Rep.'] = [('Democratic Republic of the Congo', 86)]
mig_schon['Greenland'] = None
mig_schon['Cayman Islands'] = None
mig_schon['Macao SAR, China'] = None
mig_schon['Bermuda'] = None
mig_schon['Korea, Dem. Rep.'] = [('South Korea',100)]
mig_schon['Puerto Rico'] = None
mig_schon['Serbia'] = None
mig_schon['Isle of Man'] = None

In [311]:
mig_schon_ = {'Other South': 'MISSING',
              'World': 'MISSING'}
for kk, vv in mig_schon.items():
    try:
        mig_schon_[kk]=vv[0][0]
    except:
        mig_schon_[kk] = 'MISSING'

        

In [312]:
counts = {}
for kk, vv in mig_schon_.items():
    if vv not in counts:
        counts[vv] = 1
    else:
        counts[vv] +=1

In [313]:
for kk, vv in counts.items():
    if vv > 1:
        print(kk, vv)

MISSING 25


In [314]:
mig[mig.columns[0]] = mig[mig.columns[0]].apply(lambda x: mig_schon_[x])

In [315]:
mig.rename(columns={mig.columns[0]: 'Origin'},inplace=True)

In [316]:
mig.rename(columns=mig_schon_, inplace=True)

In [317]:
mig = mig[mig['Origin']!='MISSING']

In [318]:
mig = mig[list(set(mig.columns) - set(['MISSING','Other North','Other South']))]

In [319]:
mig = mig.set_index('Origin')

In [320]:
mig = mig.reindex(sorted(mig.columns), axis=1)

In [321]:
mig = pd.DataFrame(mig.stack()).reset_index().rename(columns={'Origin': 'state.origin.name', 
                                                              'level_1': 'state.destination.name',
                                                               0: 'immigrant.population.2017'})

In [322]:
def to_float(x):
    if isinstance(x, str):
        return float(x.replace(',',''))
    else:
        return x

In [323]:
mig['immigrant.population.2017'] = mig['immigrant.population.2017'].apply(lambda x: to_float(x))

In [324]:
df = pd.merge(df, mig, how='left', left_on=['state.origin.name','state.destination.name'],
                              right_on=['state.origin.name','state.destination.name'])

In [325]:
df['immigrant.population.2017'] = df['immigrant.population.2017'].fillna(0)

In [326]:
df

,ccode1,ccode2,state.destination.name,state.destination.abb,state.origin.name,state.origin.abb,dyad.id,gdppc.2010.gradient,gdppc.2009.gradient,gdppc.2008.gradient,...,polyarchy.additive.1998.gradient,polyarchy.additive.1997.gradient,polyarchy.additive.1996.gradient,polyarchy.additive.1995.gradient,polyarchy.additive.1994.gradient,polyarchy.additive.1993.gradient,polyarchy.additive.1992.gradient,polyarchy.additive.1991.gradient,polyarchy.additive.1990.gradient,immigrant.population.2017
0,2,20,Canada,CAN,United States of America,USA,220,-928.324859,-1033.497806,-855.829272,...,-0.002045,-0.001584,-0.004737,-0.006503,-0.006503,-0.003054,-0.004590,-0.007776,-0.010695,316649.0
1,2,130,Ecuador,ECU,United States of America,USA,2130,-43716.785060,-43002.372370,-44740.460130,...,-0.066733,-0.061922,-0.072589,-0.071069,-0.069920,-0.059370,-0.058553,-0.061597,-0.063336,39038.0
2,2,160,Argentina,ARG,United States of America,USA,2160,-38097.573520,-38146.929000,-39239.385640,...,-0.017869,-0.016201,-0.017773,-0.019450,-0.017674,-0.018326,-0.015642,-0.018099,-0.018633,27093.0
3,2,220,France,FRN,United States of America,USA,2220,-7668.320563,-7456.840544,-7816.877535,...,0.017376,0.017837,0.015268,0.013282,0.012458,0.015907,0.017406,0.014862,0.011944,54724.0
4,2,225,Switzerland,SWZ,United States of America,USA,2225,25903.033720,25325.819650,26059.639830,...,0.014855,0.015316,0.013763,0.012974,0.014123,0.017572,0.017258,0.014714,0.008352,36582.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37825,316,347,Kosovo,KOS,Czech Republic,CZR,316347,-16480.751860,-16173.251930,-17354.115910,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
37826,816,516,Burundi,BUI,Vietnam,DRV,816516,-1119.352533,-1052.593413,-1000.297901,...,0.042776,0.046872,0.178634,0.189983,0.065748,0.321033,0.012497,-0.135159,-0.138943,0.0
37827,347,690,Kuwait,KUW,Kosovo,KOS,347690,34441.929270,37826.312140,43901.132990,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
37828,331,713,Taiwan,TAW,San Marino,SNM,331713,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [327]:
df.to_csv('/Users/brandonrose/repos/WM/columbia/MR-QAP-refugees/Data_5_UPDATED.csv', index=False)